In [ ]:
Symmetric Key Encryption Algorithms : These algorithms use the same key for both encryption and decryption.
Example:
AES (Advanced Encryption Standard): A widely used encryption standard, efficient and secure, with 128-, 192-, and 256-bit key options.

In [6]:
!pip install cryptography

from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes
import os
from cryptography.hazmat.primitives import padding

# Generate a random salt
def generate_salt():
    return os.urandom(16)

# Derive a 32-byte key from a password using PBKDF2 HMAC
def derive_key(password, salt):
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=100000,
        backend=default_backend()
    )
    key = kdf.derive(password.encode())
    return key

# Encrypt a file
def encrypt_file(file_path):
    # Generate a static password and derive the key
    password = "static_secure_password"
    salt = generate_salt()
    key = derive_key(password, salt)
    iv = os.urandom(16)

    # Initialize the AES cipher in CBC mode with the derived key and IV
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    encryptor = cipher.encryptor()

    # Read the file content
    with open(file_path, 'rb') as file:
        file_data = file.read()

    # Pad the file data to be AES block size compatible (128 bits or 16 bytes)
    padder = padding.PKCS7(128).padder()
    padded_data = padder.update(file_data) + padder.finalize()

    # Encrypt the data
    encrypted_data = encryptor.update(padded_data) + encryptor.finalize()

    # Save the encrypted data with salt and iv as the first 32 bytes
    encrypted_file_path = file_path + '.encrypted'
    with open(encrypted_file_path, 'wb') as encrypted_file:
        encrypted_file.write(salt + iv + encrypted_data)

    print(f"File '{file_path}' has been encrypted and saved as '{encrypted_file_path}'.")

# Decrypt a file
def decrypt_file(encrypted_file_path):
    # Generate a static password
    password = "static_secure_password"

    # Read the encrypted file content
    with open(encrypted_file_path, 'rb') as encrypted_file:
        file_content = encrypted_file.read()

    # Extract the salt, iv, and encrypted data
    salt = file_content[:16]
    iv = file_content[16:32]
    encrypted_data = file_content[32:]

    # Derive the key from the password and salt
    key = derive_key(password, salt)

    # Initialize the AES cipher in CBC mode with the derived key and IV
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    decryptor = cipher.decryptor()

    # Decrypt the data
    decrypted_padded_data = decryptor.update(encrypted_data) + decryptor.finalize()

    # Remove padding from decrypted data
    unpadder = padding.PKCS7(128).unpadder()
    decrypted_data = unpadder.update(decrypted_padded_data) + unpadder.finalize()

    # Save the decrypted data to a new file
    decrypted_file_path = encrypted_file_path.replace('.encrypted', '.decrypted')
    with open(decrypted_file_path, 'wb') as decrypted_file:
        decrypted_file.write(decrypted_data)

    print(f"File '{encrypted_file_path}' has been decrypted and saved as '{decrypted_file_path}'.")

file_path = input("Enter the path of the file to encrypt/decrypt: ")

# Encrypt the file
encrypt_file(file_path)

# Decrypt the file
decrypt_file(file_path + '.encrypted')


Enter the path of the file to encrypt/decrypt:  C:\Users\Administrator\Desktop\Sample Text.txt


File 'C:\Users\Administrator\Desktop\Sample Text.txt' has been encrypted and saved as 'C:\Users\Administrator\Desktop\Sample Text.txt.encrypted'.
File 'C:\Users\Administrator\Desktop\Sample Text.txt.encrypted' has been decrypted and saved as 'C:\Users\Administrator\Desktop\Sample Text.txt.decrypted'.


In [ ]:
 Asymmetric Key Encryption Algorithms
Overview: These use a pair of keys—a public key for encryption and a private key for decryption. Often used in secure data exchange.
Examples:
RSA (Rivest-Shamir-Adleman) : One of the most widely used algorithms for secure data transmission, based on the factorization of large integers.

In [10]:
# Install the required library
!pip install cryptography

from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.backends import default_backend

# Generate RSA keys
def generate_keys():
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048,
        backend=default_backend()
    )
    public_key = private_key.public_key()

    # Serialize the private key
    private_pem = private_key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.TraditionalOpenSSL,
        encryption_algorithm=serialization.NoEncryption()
    )

    # Serialize the public key
    public_pem = public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )

    with open("private_key.pem", "wb") as private_file:
        private_file.write(private_pem)

    with open("public_key.pem", "wb") as public_file:
        public_file.write(public_pem)

    print("Keys generated and saved to files.")

# Encrypt a file using the public key
def encrypt_file(file_path: str, public_key_path: str):
    with open(file_path, 'rb') as file:
        data = file.read()

    with open(public_key_path, 'rb') as key_file:
        public_key = serialization.load_pem_public_key(
            key_file.read(),
            backend=default_backend()
        )

    encrypted_data = public_key.encrypt(
        data,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )

    with open(file_path + '.enc', 'wb') as encrypted_file:
        encrypted_file.write(encrypted_data)

    print(f'File {file_path} encrypted successfully.')

# Decrypt a file using the private key
def decrypt_file(file_path: str, private_key_path: str):
    with open(file_path, 'rb') as encrypted_file:
        encrypted_data = encrypted_file.read()

    with open(private_key_path, 'rb') as key_file:
        private_key = serialization.load_pem_private_key(
            key_file.read(),
            password=None,
            backend=default_backend()
        )

    decrypted_data = private_key.decrypt(
        encrypted_data,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )

    output_file_path = file_path.replace('.enc', '_decrypted')
    with open(output_file_path, 'wb') as decrypted_file:
        decrypted_file.write(decrypted_data)

    print(f'File decrypted successfully to {output_file_path}.')

file_path = input("Enter the path of the file to encrypt/decrypt: ")

# Generate RSA keys
generate_keys()

# Encrypt the file
encrypt_file(file_path, 'public_key.pem')

# Decrypt the file
decrypt_file(file_path + '.enc', 'private_key.pem')


Enter the path of the file to encrypt/decrypt:  C:\Users\Administrator\Desktop\sample.txt


Keys generated and saved to files.
File C:\Users\Administrator\Desktop\sample.txt encrypted successfully.
File decrypted successfully to C:\Users\Administrator\Desktop\sample.txt_decrypted.


In [ ]:
Hash Functions
Overview: Not strictly encryption, as they are one-way functions that map data to a fixed-size hash value, primarily used for data integrity.
Examples:
SHA (Secure Hash Algorithm): Commonly used for integrity checks, with variations like SHA-1, SHA-256, SHA-512.

In [12]:
import hashlib

def hash_file(file_path: str, hash_algorithm: str = 'sha256') -> str:
    # Create a hash object
    hash_func = hashlib.new(hash_algorithm)

    # Open the file in binary mode and read it in chunks
    with open(file_path, 'rb') as file:
        while chunk := file.read(4096):
            hash_func.update(chunk)

    # Return the hexadecimal representation of the hash
    return hash_func.hexdigest()

def verify_file(file_path: str, expected_hash: str, hash_algorithm: str = 'sha256') -> bool:
    # Hash the file
    file_hash = hash_file(file_path, hash_algorithm)

    # Compare the computed hash with the expected hash
    return file_hash == expected_hash

file_path = input("Enter the path of the file to hash and verify: ")

# Hash the file
file_hash = hash_file(file_path)
print(f'Hash of the file: {file_hash}')

# Verify the file
is_verified = verify_file(file_path, file_hash)
print(f'File verification: {"Success" if is_verified else "Failure"}')


Enter the path of the file to hash and verify:  C:\Users\Administrator\Desktop\Sample Text.txt


Hash of the file: 803a3698523f0001ba7410eb90b1364d8cf66e1d081543a4eb8703f95d6c2e99
File verification: Success


In [ ]:
Stream Ciphers
Overview: Encrypts data one bit or byte at a time, often faster than block ciphers for real-time applications.
Examples:
RC4: Though now considered insecure, it was widely used in protocols like WEP and SSL.

In [14]:
# Install the required library
!pip install cryptography

from cryptography.hazmat.primitives.ciphers import Cipher, algorithms
from cryptography.hazmat.backends import default_backend
import os

def generate_key():
    return os.urandom(32)  # Generates a 256-bit key

def encrypt_file(file_path: str, key: bytes):
    # Read the file contents
    with open(file_path, 'rb') as file:
        data = file.read()

    # Generate a random nonce
    nonce = os.urandom(16)

    # Initialize the cipher
    algorithm = algorithms.ChaCha20(key, nonce)
    cipher = Cipher(algorithm, mode=None, backend=default_backend())
    encryptor = cipher.encryptor()

    # Encrypt the data
    encrypted_data = encryptor.update(data)

    # Save the encrypted data and nonce
    with open(file_path + '.enc', 'wb') as encrypted_file:
        encrypted_file.write(nonce + encrypted_data)

    print(f'File {file_path} encrypted successfully.')

def decrypt_file(file_path: str, key: bytes):
    # Read the encrypted file contents
    with open(file_path, 'rb') as encrypted_file:
        nonce = encrypted_file.read(16)  # First 16 bytes are the nonce
        encrypted_data = encrypted_file.read()

    # Initialize the cipher
    algorithm = algorithms.ChaCha20(key, nonce)
    cipher = Cipher(algorithm, mode=None, backend=default_backend())
    decryptor = cipher.decryptor()

    # Decrypt the data
    decrypted_data = decryptor.update(encrypted_data)

    # Save the decrypted data
    output_file_path = file_path.replace('.enc', '_decrypted')
    with open(output_file_path, 'wb') as decrypted_file:
        decrypted_file.write(decrypted_data)

    print(f'File decrypted successfully to {output_file_path}.')

file_path = input("Enter the path of the file to encrypt/decrypt: ")

# Generate the key
key = generate_key()

# Encrypt the file
encrypt_file(file_path, key)

# Decrypt the file
decrypt_file(file_path + '.enc', key)


Enter the path of the file to encrypt/decrypt:  C:\Users\Administrator\Desktop\Sample Text.txt


File C:\Users\Administrator\Desktop\Sample Text.txt encrypted successfully.
File decrypted successfully to C:\Users\Administrator\Desktop\Sample Text.txt_decrypted.


In [ ]:
Password-Based Encryption (PBE) Algorithms
Overview: Algorithms that derive encryption keys from user-provided passwords using key derivation functions (KDFs).
Examples:
PBKDF2 (Password-Based Key Derivation Function 2): Uses a salt and iteration count to derive secure encryption keys from passwords.

In [80]:
# Install the required library
!pip install cryptography

from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.backends import default_backend
import os

def derive_key(password: str, salt: bytes) -> bytes:
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=100000,
        backend=default_backend()
    )
    key = kdf.derive(password.encode())
    return key

def encrypt_file(file_path: str, password: str):
    # Read the file contents
    with open(file_path, 'rb') as file:
        data = file.read()

    # Generate a random salt and IV (Initialization Vector)
    salt = os.urandom(16)
    iv = os.urandom(16)

    # Derive the key from the password and salt
    key = derive_key(password, salt)

    # Initialize the cipher
    cipher = Cipher(algorithms.AES(key), modes.CFB(iv), backend=default_backend())
    encryptor = cipher.encryptor()

    # Encrypt the data
    encrypted_data = encryptor.update(data) + encryptor.finalize()

    # Save the encrypted data, salt, and IV
    with open(file_path + '.enc', 'wb') as encrypted_file:
        encrypted_file.write(salt + iv + encrypted_data)

    print(f'File {file_path} encrypted successfully.')

def decrypt_file(file_path: str, password: str):
    # Read the encrypted file contents
    with open(file_path, 'rb') as encrypted_file:
        encrypted_data = encrypted_file.read()

    # Extract the salt, IV, and encrypted data
    salt = encrypted_data[:16]
    iv = encrypted_data[16:32]
    ciphertext = encrypted_data[32:]

    # Derive the key from the password and salt
    key = derive_key(password, salt)

    # Initialize the cipher
    cipher = Cipher(algorithms.AES(key), modes.CFB(iv), backend=default_backend())
    decryptor = cipher.decryptor()

    # Decrypt the data
    decrypted_data = decryptor.update(ciphertext) + decryptor.finalize()

    # Save the decrypted data
    output_file_path = file_path.replace('.enc', '_decrypted')
    with open(output_file_path, 'wb') as decrypted_file:
        decrypted_file.write(decrypted_data)

    print(f'File decrypted successfully to {output_file_path}.')

file_path = input("Enter the path of the file to encrypt/decrypt: ")

# Encrypt the file
encrypt_file(file_path, 'my_secure_password')

# Decrypt the file
decrypt_file(file_path + '.enc', 'my_secure_password')


Enter the path of the file to encrypt/decrypt:  C:\\Users\\Administrator\\Desktop\\Sample Text.txt


File C:\\Users\\Administrator\\Desktop\\Sample Text.txt encrypted successfully.
File decrypted successfully to C:\\Users\\Administrator\\Desktop\\Sample Text.txt_decrypted.
